# Artur Kompała

Celem zadania jest stworzenie modelu do prognozowania przyszłych cen akcji na podstawie historycznych danych giełdowych z użyciem sieci rekurencyjnej (RNN lub LSTM).

dane do wykrozystania:

https://www.kaggle.com/datasets/jacksoncrow/stock-market-dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# 1. Wczytanie danych z pliku
df = pd.read_csv('data/NVDA.csv')

# 2. Czyszczenie i przygotowanie danych
# Konwersja daty i sortowanie
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.sort_values('Date')

# Upewnienie się, że kolumna Close jest numeryczna
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

# Usunięcie ewentualnych pustych wierszy (ważne, aby uniknąć błędów 'nan' w modelu)
df = df.dropna(subset=['Date', 'Close'])

# Wybranie tylko kolumny z ceną zamknięcia
data = df.filter(['Close'])
dataset = data.values

# 3. Normalizacja danych (skalowanie do przedziału 0-1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# Ustawienie długości sekwencji (na podstawie ilu dni przewidujemy kolejny)
prediction_days = 60

# Podział na zbiór treningowy (80%) i testowy
training_data_len = int(np.ceil(len(dataset) * 0.8))

train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []

for i in range(prediction_days, len(train_data)):
    x_train.append(train_data[i-prediction_days:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# 4. Budowa modelu LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Trenowanie modelu
print("Rozpoczynam trening modelu...")
model.fit(x_train, y_train, batch_size=32, epochs=5)

# 5. Przygotowanie danych testowych i predykcja
test_data = scaled_data[training_data_len - prediction_days:, :]
x_test = []
y_test = dataset[training_data_len:, :]

for i in range(prediction_days, len(test_data)):
    x_test.append(test_data[i-prediction_days:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Generowanie prognoz
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# 6. Wizualizacja wyników
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(16, 8))
plt.title('Model LSTM - Prognoza cen akcji NVDA')
plt.xlabel('Data', fontsize=18)
plt.ylabel('Cena Zamknięcia USD', fontsize=18)
plt.plot(train['Close'], label='Dane Treningowe')
plt.plot(valid['Close'], label='Cena Rzeczywista')
plt.plot(valid['Predictions'], label='Prognoza Modelu')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

ModuleNotFoundError: No module named 'tensorflow.python'